In [164]:
from decouple import config
from langchain_openai import ChatOpenAI

SECRET_KEY = config('OPENAI_API_KEY')
chat = ChatOpenAI(openai_api_key=SECRET_KEY)

In [176]:
# from langchain_core.messages import HumanMessage, SystemMessage

# messages = [
#     SystemMessage(content="You're a helpful assistant"),
#     HumanMessage(content="What is Laravel?"),
# ]

In [177]:
# response = chat.invoke(messages)

In [179]:
# print(response.content)
# chat.batch([messages])

[AIMessage(content='Laravel is a popular open-source PHP framework used for building web applications. It provides a clean and elegant syntax while also offering a wide range of features that help developers quickly and efficiently create robust web applications. Some key features of Laravel include a powerful ORM (Object-Relational Mapping) system, built-in authentication and authorization, routing, caching, and more. Laravel follows the MVC (Model-View-Controller) architectural pattern, making it easy to organize code and separate concerns. It also has a vibrant community and extensive documentation, making it a popular choice for many web developers.')]

In [168]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.codenterprise.com")
docs = loader.load()

In [169]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=0
)

documents  = text_splitter.split_documents(docs)

Created a chunk of size 1244, which is longer than the specified 250
Created a chunk of size 722, which is longer than the specified 250
Created a chunk of size 763, which is longer than the specified 250
Created a chunk of size 315, which is longer than the specified 250
Created a chunk of size 326, which is longer than the specified 250
Created a chunk of size 503, which is longer than the specified 250
Created a chunk of size 278, which is longer than the specified 250
Created a chunk of size 278, which is longer than the specified 250
Created a chunk of size 261, which is longer than the specified 250
Created a chunk of size 502, which is longer than the specified 250
Created a chunk of size 501, which is longer than the specified 250
Created a chunk of size 307, which is longer than the specified 250
Created a chunk of size 361, which is longer than the specified 250
Created a chunk of size 297, which is longer than the specified 250
Created a chunk of size 297, which is longer th

In [170]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [171]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    SystemMessage(content="You're a helpful assistant"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

# document_chain = create_stuff_documents_chain(chat, prompt)

In [172]:
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever

retriever = db.as_retriever()

# retrieval_chain = create_retrieval_chain(retriever, document_chain)
retriever_chain = create_history_aware_retriever(chat, retriever, prompt)


In [175]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    # HumanMessage(content="Who is Atif ali?"), 
    # AIMessage(content="Yes!")
]

response = retriever_chain.invoke({
    "chat_history": chat_history, 
    "input": "List me the names of Software enginners?"
    })

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(chat, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

chat_history = [HumanMessage(content="List me the names of Software enginners?"), AIMessage(content="Yes!")]
response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me the list of all"
})

# from langchain_core.messages import HumanMessage, AIMessage

# chat_history = [HumanMessage(content="Who is Atif ali?"), AIMessage(content="Yes!")]
# response = retriever_chain.invoke({
#     "chat_history": chat_history,
#     "input": "Who is Atif ali?"
# })

pprint(response['answer'])

'The CTO at Codenterprise is [Name].'
